## AstroComputer sample in Scala

Add the required `jar-file` to your classpath. You must have run
```
$ ../../gradlew clean build
```
This one will generate the `/../build/libs/AstroComputer-1.0.jar`  
or
```
$ ../../gradlew clean shadowJar
```
This one will generate the `/../build/libs/AstroComputer-1.0-all.jar`  
For this last one, you need to tweak the shadowJar plugin definition.  
To do so, modify the gradle script so it says
```
plugins {
  ...
  id 'com.github.johnrengelman.shadow' version '6.0.0' // Remove version if used in a gradle submodule.
  ...
}
```

In [7]:
import java.io.File

val absolutePath = os.pwd + "/../build/libs/AstroComputer-1.0.jar" // Depends on which file you've generated with Gradle, above.
val jarFile = new File(absolutePath)
println(s"File exists: ${jarFile.exists()}")
var cp = repl.sess.frames.head.classpath;
println(s"ClassPath: ${cp}")

// val path = ammonite.ops.Path(absolutePath)
// interp.load.cp(path)

interp.load.cp(os.Path(absolutePath))
cp = repl.sess.frames.head.classpath;
println(s"ClassPath now: ${cp}")


File exists: true
ClassPath: List(file:/Users/olivierlediouris/repos/raspberry-coffee/astro-computer/AstroComputer/build/libs/AstroComputer-1.0.jar)
ClassPath now: List(file:/Users/olivierlediouris/repos/raspberry-coffee/astro-computer/AstroComputer/build/libs/AstroComputer-1.0.jar, file:/Users/olivierlediouris/repos/raspberry-coffee/astro-computer/AstroComputer/build/libs/AstroComputer-1.0.jar)


import java.io.File


absolutePath: String = "/Users/olivierlediouris/repos/raspberry-coffee/astro-computer/AstroComputer/notebooks/../build/libs/AstroComputer-1.0.jar"
jarFile: File = /Users/olivierlediouris/repos/raspberry-coffee/astro-computer/AstroComputer/notebooks/../build/libs/AstroComputer-1.0.jar
cp: Seq[java.net.URL] = List(
  file:/Users/olivierlediouris/repos/raspberry-coffee/astro-computer/AstroComputer/build/libs/AstroComputer-1.0.jar,
  file:/Users/olivierlediouris/repos/raspberry-coffee/astro-computer/AstroComputer/build/libs/AstroComputer-1.0.jar
)

Now we can get started.  
First, `imports`:

In [8]:
import java.text.SimpleDateFormat
import java.util.{Calendar, TimeZone}
import astro.celestial.LongTermAlmanac
import astro.utils.MiscUtils.{decToSex, lpad, renderEoT, renderRA, renderSdHp}
import astro.utils.{MiscUtils, TimeUtils}
import astro.celestial.Core.{moonPhase, weekDay}

import java.text.SimpleDateFormat

import java.util.{Calendar, TimeZone}

import astro.celestial.LongTermAlmanac

import astro.utils.MiscUtils.{decToSex, lpad, renderEoT, renderRA, renderSdHp}

import astro.utils.{MiscUtils, TimeUtils}

import astro.celestial.Core.{moonPhase, weekDay}

Date masks settings:

In [9]:
val SDF_UTC = new SimpleDateFormat("yyyy-MM-dd HH:mm:ss 'UTC'")
SDF_UTC.setTimeZone(TimeZone.getTimeZone("Etc/UTC"))

SDF_UTC: SimpleDateFormat = java.text.SimpleDateFormat@658bc94a

Now for real, set the date to March 28, 2020 at 16:50:20 UTC:

In [10]:
val date = Calendar.getInstance(TimeZone.getTimeZone("Etc/UTC")) // Now
date.set(Calendar.YEAR, 2020)
date.set(Calendar.MONTH, Calendar.MARCH)
date.set(Calendar.DAY_OF_MONTH, 28)
date.set(Calendar.HOUR_OF_DAY, 16) // and not just Calendar.HOUR !!!!
date.set(Calendar.MINUTE, 50)
date.set(Calendar.SECOND, 20)
date.set(Calendar.MILLISECOND, 0)

println(String.format("Calculations for %s", SDF_UTC.format(date.getTime)))


Calculations for 2020-03-28 16:50:20 UTC


date: Calendar = java.util.GregorianCalendar[time=1585414220000,areFieldsSet=true,areAllFieldsSet=false,lenient=true,zone=sun.util.calendar.ZoneInfo[id="Etc/UTC",offset=0,dstSavings=0,useDaylight=false,transitions=0,lastRule=null],firstDayOfWeek=1,minimalDaysInFirstWeek=1,ERA=1,YEAR=2020,MONTH=2,WEEK_OF_YEAR=13,WEEK_OF_MONTH=4,DAY_OF_MONTH=28,DAY_OF_YEAR=88,DAY_OF_WEEK=7,DAY_OF_WEEK_IN_MONTH=4,AM_PM=1,HOUR=4,HOUR_OF_DAY=16,MINUTE=50,SECOND=20,MILLISECOND=0,ZONE_OFFSET=0,DST_OFFSET=0]

Date is set, we trigger the calculation (with dynamic DeltaT)

In [11]:
// Take time here, JVM is loaded, etc...
val before = System.currentTimeMillis
// Recalculate
val deltaT = TimeUtils.getDeltaT(date.get(Calendar.YEAR), date.get(Calendar.MONTH) + 1)
println(s"New deltaT: $deltaT")
// All calculations here
val result = LongTermAlmanac.calculate(
  date.get(Calendar.YEAR),
  date.get(Calendar.MONTH) + 1, // Yes, +1. Jan: 1, Dec: 12.
  date.get(Calendar.DAY_OF_MONTH),
  date.get(Calendar.HOUR_OF_DAY),
  date.get(Calendar.MINUTE),
  date.get(Calendar.SECOND),
  deltaT)
val after = System.currentTimeMillis
// Done with calculations, ready for display

New deltaT: 71.71293632812495


before: Long = 1660206614213L
deltaT: Double = 71.71293632812495
result: astro.celestial.AstroContext = astro.celestial.AstroContext@706f967f
after: Long = 1660206614217L

Done. Now, display results.

In [12]:
println(s"Calculations done for ${SDF_UTC.format(date.getTime)}, in ${after - before} ms <<<")
println(s"Sun:\t Decl: ${lpad(decToSex(result.DECsun, MiscUtils.SWING, MiscUtils.NS), 10, " ")}, " +
  s"GHA: ${lpad(decToSex(result.GHAsun, MiscUtils.SWING, MiscUtils.NONE), 11, " ")}, " +
  s"RA: ${renderRA(result.RAsun)}, " +
  s"SD: ${lpad(renderSdHp(result.SDsun), 9, " ")}, " +
  s"HP: ${lpad(renderSdHp(result.HPsun), 9, " ")}")
println(s"Moon:\t Decl: ${lpad(decToSex(result.DECmoon, MiscUtils.SWING, MiscUtils.NS), 10, " ")}, " +
  s"GHA: ${lpad(decToSex(result.GHAmoon, MiscUtils.SWING, MiscUtils.NONE), 11, " ")}, " +
  s"RA: ${renderRA(result.RAmoon)}, " +
  s"SD: ${lpad(renderSdHp(result.SDmoon), 9, " ")}, " +
  s"HP: ${lpad(renderSdHp(result.HPmoon), 9, " ")}")
println(s"Venus:\t Decl: ${lpad(decToSex(result.DECvenus, MiscUtils.SWING, MiscUtils.NS), 10, " ")}, " +
  s"GHA: ${lpad(decToSex(result.GHAvenus, MiscUtils.SWING, MiscUtils.NONE), 11, " ")}, " +
  s"RA: ${renderRA(result.RAvenus)}, " +
  s"SD: ${lpad(renderSdHp(result.SDvenus), 9, " ")}, " +
  s"HP: ${lpad(renderSdHp(result.HPvenus), 9, " ")}")
println(s"Mars:\t Decl: ${lpad(decToSex(result.DECmars, MiscUtils.SWING, MiscUtils.NS), 10, " ")}, " +
  s"GHA: ${lpad(decToSex(result.GHAvenus, MiscUtils.SWING, MiscUtils.NONE), 11, " ")}, " +
  s"RA: ${renderRA(result.RAmars)}, " +
  s"SD: ${lpad(renderSdHp(result.SDmars), 9, " ")}, " +
  s"HP: ${lpad(renderSdHp(result.HPmars), 9, " ")}")
println(s"Jupiter:\t Decl: ${lpad(decToSex(result.DECjupiter, MiscUtils.SWING, MiscUtils.NS), 10, " ")}, " +
  s"GHA: ${lpad(decToSex(result.GHAjupiter, MiscUtils.SWING, MiscUtils.NONE), 11, " ")}, " +
  s"RA: ${renderRA(result.RAjupiter)}, " +
  s"SD: ${lpad(renderSdHp(result.SDjupiter), 9, " ")}, " +
  s"HP: ${lpad(renderSdHp(result.HPjupiter), 9, " ")}")
println(s"Saturn:\t Decl: ${lpad(decToSex(result.DECsaturn, MiscUtils.SWING, MiscUtils.NS), 10, " ")}, " +
  s"GHA: ${lpad(decToSex(result.GHAsaturn, MiscUtils.SWING, MiscUtils.NONE), 11, " ")}, " +
  s"RA: ${renderRA(result.RAsaturn)}, " +
  s"SD: ${lpad(renderSdHp(result.SDsaturn), 9, " ")}, " +
  s"HP: ${lpad(renderSdHp(result.HPsaturn), 9, " ")}")
println("")
println(s"Polaris:\t Decl: ${lpad(decToSex(result.DECpol, MiscUtils.SWING, MiscUtils.NS), 10, " ")}, " +
  s"GHA: ${lpad(decToSex(result.GHApol, MiscUtils.SWING, MiscUtils.NONE), 11, " ")}, " +
  s"RA: ${renderRA(result.RApol)} ")
println(s"Equation of Time: ${renderEoT(result.EoT)}")
println(s"Lunar Distance: ${lpad(decToSex(result.LDist, MiscUtils.SWING, MiscUtils.NONE), 10, " ")}")
println(s"Moon Phase: ${moonPhase(result)}")
println(s"Day of Week: ${LongTermAlmanac.WEEK_DAYS(weekDay(result))}")
println
println("Done with Scala!")


Calculations done for 2020-03-28 16:50:20 UTC, in 4 ms <<<
Sun:	 Decl:  3°21.51'N, GHA:   71°21.81', RA: 00h 31m 07.00s, SD: 16'01.22“, HP:    08.81“
Moon:	 Decl: 16°11.40'N, GHA:   25°09.65', RA: 03h 35m 56.00s, SD: 14'54.36“, HP: 54'42.28“
Venus:	 Decl: 22°06.81'N, GHA:   27°59.36', RA: 03h 24m 37.00s, SD:    12.38“, HP:    12.94“
Mars:	 Decl: 21°20.44'S, GHA:   27°59.36', RA: 20h 03m 22.00s, SD:    03.15“, HP:    05.91“
Jupiter:	 Decl: 21°22.04'S, GHA:  143°16.47', RA: 19h 43m 29.00s, SD:    18.32“, HP:    01.64“
Saturn:	 Decl: 20°06.29'S, GHA:  136°28.12', RA: 20h 10m 42.00s, SD:    07.99“, HP:    00.85“

Polaris:	 Decl: 89°21.04'N, GHA:   35°17.04', RA: 02h 55m 26.00s 
Equation of Time: - 04m 60.0s
Lunar Distance:  47°10.07'
Moon Phase:  +cre
Day of Week: SAT

Done with Scala!
